In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!pip install pandas
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd ..
!rm -rf community-events
!pip install wandb
#!huggingface-cli login
#!wandb login
#!accelerate config

In [4]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os


'''
Reference parameters 

hyperparams_cyclegan = {
    'decay_epoch':[10] , #100
    'lr':[0.0002], 
    'n_residual_blocks':[9], 
    
    'lambda_cyc':[10.0], 
    'lambda_id':[5.0], 
    'beta1':[0.5], 
    'beta2':[0.999], 
    'batch_size':[8],
    
    'dataset_name':['Chris1/sim2real_gta5_to_cityscapes'],#['Chris1/NFT_cryptopunks_to_bored_apes'],  

    'channels':[3], 
    'checkpoint_interval':[10], 
    #'cpu':[False],     
    'epoch':[0], 
    #'fp16':[False], 
    'image_size':[256], 
    'mixed_precision':['no'], 
    'num_epochs':[20], 
    'num_workers':[8], 
    'organization_name':['Chris1'], 
    
    'wandb':[True],
    'push_to_hub':[True], 
    'model_name':['test'],
    
    'pytorch_dump_folder_path':[None], 
    'sample_interval':[100]
                     }
'''
hyperparams_cyclegan = { 
                        'lr':[0.0002, 0.0004, 0.0008, 0.002], 
                        'num_epochs':[100], 
                        'decay_epoch':[10, 30, 50, 70] , #100 
                        'n_residual_blocks':[9], 
                        'lambda_cyc':[10.0], 'lambda_id':[5.0], 
                        'beta1':[0.5], 'beta2':[0.999], 
                        'batch_size':[32], 
                        'dataset_name':['Chris1/sim2real_gta5_to_cityscapes'],#['Chris1/NFT_cryptopunks_to_bored_apes'], 
                        'channels':[3], 
                        'checkpoint_interval':[10], #'cpu':[False], 
                        'epoch':[0], #'fp16':[False], 
                        'image_size':[64], #256
                        'mixed_precision':['no'], 
                        'num_workers':[8], 
                        'organization_name':['Chris1'], 
                        'wandb':[True],
                        'push_to_hub':[True], 
                        'model_name':['test_cyclegan'],
                        'pytorch_dump_folder_path':['torch_dump'], 
                        'sample_interval':[100] }


In [6]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)



for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df = experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments_cyclegan/{experiment_id}'
        grid['pytorch_dump_folder_path'] = f'{grid["pytorch_dump_folder_path"]}_{experiment_id}'
        grid['model_name'] = f'{grid["model_name"]}_{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
        call(call_params)
        
        
        experiments_df.to_csv(EXPERIMENTS_DATA_CSV)

   batch_size  beta1  beta2  channels  checkpoint_interval  \
0        32.0    0.5  0.999       3.0                 10.0   

                         dataset_name  decay_epoch  epoch  image_size  \
0  Chris1/sim2real_gta5_to_cityscapes         10.0    0.0       128.0   

   lambda_cyc  ...  model_name  n_residual_blocks num_epochs num_workers  \
0        10.0  ...        test                9.0      100.0         8.0   

   organization_name                                         output_dir  \
0             Chris1  batch_size__32--beta1__0.5--beta2__0.999--chan...   

   push_to_hub pytorch_dump_folder_path sample_interval  wandb  
0          1.0                     None           100.0    1.0  

[1 rows x 23 columns]
   batch_size  beta1  beta2  channels  checkpoint_interval  \
0        32.0    0.5  0.999       3.0                 10.0   
1        32.0    0.5  0.999       3.0                 10.0   

                         dataset_name  decay_epoch  epoch  image_size  \
0  Chris1/s